# Библиотеки  

In [53]:
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# Импортируем Dataset

In [54]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

# Вычисляем TF-IDF признаки для всех текстов

## Само TF-IDF

In [55]:
vectorizer = TfidfVectorizer()

In [56]:
X = vectorizer.fit_transform(newsgroups.data)

In [57]:
Y = newsgroups.target

# Подбираем наилучший параметр С для SVC

## Введем кросс-валидацию

In [58]:
cv = KFold(shuffle=True,random_state=421) # Возможно random_state=42

## Подбор параметров

In [59]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
# cv = KFold(y.size, n_folds=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=421, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [60]:
for a in gs.grid_scores_:
    a.mean_validation_score
    a.parameters

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [61]:
c13 = gs.best_params_
print(c13)

{'C': 10.0}


## Теперь обучаем модель по найденному параметру C

In [62]:
model1 = SVC(kernel='linear', C=10,random_state=241)

In [63]:
model1.fit(X, Y)

SVC(C=10, kernel='linear', random_state=241)

## Получаем коэффициенты

In [64]:
coefs = abs(model1.coef_.todense().A1)
coefs = np.argsort(coefs) # Теперь мы получаем numpy массив, 
                          # где вместо каждого значения стоит его индекс в порядке возрастания элементов

In [65]:
coefs[:-11:-1]

array([24019, 12871,  5088,  5093, 17802, 23673, 21850,  5776, 15606,
       18430], dtype=int64)

## Если надо будет найти слово по его индексу

In [70]:
feature_mapping = vectorizer.get_feature_names()
a = []
for i in range(-10,0):
    a.append(feature_mapping[coefs[i]]) # Вместо i подставьте нужный индекс

In [71]:
a.sort()

In [72]:
a

['atheism',
 'atheists',
 'bible',
 'god',
 'keith',
 'moon',
 'nick',
 'religion',
 'sky',
 'space']